<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfPBCGLDxsK-f0Wnhf9jOj4HnYDCAA8QFvPw&s" width="200">

# 01MIAR - Actividad Dataset

Esta actividad tiene el objetivo de aplicar los conocimientos adquiridos en el curso sobre tratamiento, análisis y visualización de datos con Python.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import zipfile
import io

# 1. Conjunto de datos

El dataset elegido para esta actividad contiene datos sobre más de 300.000 accidentes de tráfico en Reino Unido entre los años 2021 y 2022. Este conjunto de datos contiene información muy valiosa sobre la seguridad vial, pero a partir de ellos no vamos a poder extraer algunas conclusiones muy valiosas como por ejemplo los factores de riesgo que pueden provocar un accidente. Entre estos factores podríamos valorar las condiciones climáticas, la ubicación geográfica o el tipo de vehículo. Sin embargo, este dataset solo contiene observaciones de accidentes, por lo que no disponemos de un contrafactual para comparar con situaciones que no hayan llevado a un accidente. Por ejemplo, para estimar los factores de riesgo en el caso de las condiciones climáticas, necesitaríamos conocer todas los desplazamientos realizados en Reino Unido entre esos años, y las condiciones climáticas en cada uno, lo que nos proporcionaría una estimación válida de si una condición climática concreta provoca realmente accidentes o no. Sin embargo, si que podemos realizar otros análisis muy interesantes, como los que se describen en la siguiente sección.


## 1.1. Objetivos

Con este conjunto de datos realizaremos los siguientes análisis:

1. Estimación de factores de riesgo de accidentes graves en comparación con accidentes leves
2. Comparación del riesgo de accidentes entre regiones (valor estadístico limitado)
    - Asumiremos la hipótesis de que el riesgo base de accidentes es constante, y proporcional a la población de cada región
    - Para este cálculo tenemos que incluir datos de población, que obtendremos de ...
3. Análisis descriptivos sobre accidentes
    - Tendencia en el tiempo
    - Condiciones climáticas más comunes cuando se producen accidentes
    - Lugares más comunes donde se producen accidentes
    - etc

## 1.2. Fuente

El conjunto de datos elegido para esta actividad proviene de kaggle. El enlace para este dataset es el siguiente: [enlace al dataset road-accidents-dataset en kaggle](https://www.kaggle.com/datasets/atharvasoundankar/road-accidents-dataset). 


## 1.3. Lectura del conjunto de datos

In [39]:
# Lectura del dataset

# Petición a kaggle del dataset comprimido
url = "https://www.kaggle.com/api/v1/datasets/download/atharvasoundankar/road-accidents-dataset"
response = requests.get(url)


# Descomprimir y leer con pandas
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open("Road Accident Data.csv") as accidents_file:
        accidents = pd.read_csv(accidents_file)

In [40]:
# Mostrar contenido y variables
print(accidents.shape)
print(accidents.columns)
display(accidents.sample(5))

(307973, 21)
Index(['Accident_Index', 'Accident Date', 'Day_of_Week', 'Junction_Control',
       'Junction_Detail', 'Accident_Severity', 'Latitude', 'Light_Conditions',
       'Local_Authority_(District)', 'Carriageway_Hazards', 'Longitude',
       'Number_of_Casualties', 'Number_of_Vehicles', 'Police_Force',
       'Road_Surface_Conditions', 'Road_Type', 'Speed_limit', 'Time',
       'Urban_or_Rural_Area', 'Weather_Conditions', 'Vehicle_Type'],
      dtype='object')


,Accident_Index,Accident Date,Day_of_Week,Junction_Control,Junction_Detail,Accident_Severity,Latitude,Light_Conditions,Local_Authority_(District),Carriageway_Hazards,...,Number_of_Casualties,Number_of_Vehicles,Police_Force,Road_Surface_Conditions,Road_Type,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type
287216,BS0287217,21-09-2022,Tuesday,Give way or uncontrolled,Mini-roundabout,Slight,50.820956,Daylight,Arun,NaN,...,1,1,Sussex,Dry,Single carriageway,40,15:47,Urban,Fine no high winds,Car
206793,BS0206794,14-07-2022,Wednesday,Data missing or out of range,Not at junction or within 20 metres,Slight,55.207222,Daylight,Northumberland,NaN,...,1,2,Northumbria,Wet or damp,Single carriageway,60,10:05,Rural,Fog or mist,Car
243505,BS0243506,10-02-2022,Wednesday,Data missing or out of range,Not at junction or within 20 metres,Slight,52.978037,Daylight,Rushcliffe,NaN,...,3,2,Nottinghamshire,Frost or ice,Single carriageway,60,10:18,Rural,Other,Car
71984,BS0071985,27-03-2021,Friday,Data missing or out of range,Not at junction or within 20 metres,Slight,52.982101,Daylight,Stoke-on-Trent,NaN,...,1,2,Staffordshire,Wet or damp,Dual carriageway,50,12:07,Urban,Fine no high winds,Other vehicle
167300,BS0167301,04-11-2022,Thursday,Data missing or out of range,Not at junction or within 20 metres,Slight,51.533700,Daylight,Islington,NaN,...,1,3,Metropolitan Police,Dry,Dual carriageway,30,14:39,Urban,Fine no high winds,Car


## 1.3 Contenido del conjunto de datos

Como podemos ver en las salidas del código anteriores, este conjunto de datos tiene 21 columnas y 307.973 filas. Para describir el contenido de cada variable, las vamos dividir en dos grupos según sean categóricas o numéricas. La mayoría de ellas son categóricas, como se puede ver a continuación:

In [69]:
# Tipos de las variables y su contenido (rango, tipos de valores, NAs)

for var in accidents.columns:
    dtype = str(accidents[var].dtype)
    
    # Comentario por tipo de variable categóricas / numéricas
    if dtype == "object":
        unique_values = np.unique(accidents[var].dropna())
        comment_on_variable = f"{unique_values.shape[0]:_} valores posibles: {', '.join(unique_values[:2])}, ..."
    else:
        comment_on_variable = f"Rango: [{accidents[var].min()}, {accidents[var].max()}]"

    is_na = accidents[var].isna()
    if any(is_na):        
        comment_on_variable = f"Contiene {is_na.sum():_} NA; " + comment_on_variable

    print(f"{var:<30} -> {dtype:<10} ({comment_on_variable})")

Accident_Index                 -> object     (307_973 valores posibles: BS0000001, BS0000002, ...)
Accident Date                  -> object     (730 valores posibles: 01-01-2021, 01-01-2022, ...)
Day_of_Week                    -> object     (7 valores posibles: Friday, Monday, ...)
Junction_Control               -> object     (7 valores posibles: Authorised person, Auto traffic sigl, ...)
Junction_Detail                -> object     (9 valores posibles: Crossroads, Mini-roundabout, ...)
Accident_Severity              -> object     (4 valores posibles: Fatal, Fetal, ...)
Latitude                       -> float64    (Rango: [49.914488, 60.598055])
Light_Conditions               -> object     (5 valores posibles: Darkness - lighting unknown, Darkness - lights lit, ...)
Local_Authority_(District)     -> object     (422 valores posibles: Aberdeen City, Aberdeenshire, ...)
Carriageway_Hazards            -> object     (Contiene 302_549 NA; 5 valores posibles: Any animal in carriageway (except

#### Variables categóricas

1. ``Accident_Index``: Código identificativo con 2 letras y 7 números
2. ``Accident Date``: Fecha del accidente en formato día-mes-año.
3. ``Day_of_Week``: Día de la semana del accidente.
4. ``Junction_Control``: Tipo de regulación de la intersección del accidente (si aplica).
5. ``Junction_Detail``: Detalles sobre la intersección del accidente (si aplica).
6. ``Accident_Severity``: Tipo de gravedad del accidente
7. ``Light_Conditions``: Condiciones de luminosidad del accidente.
8. ``Local_Authority_(District)``: Distrito competente en el lugar del accidente.
9. ``Carriageway_Hazards``: Peligros en el lugar del accidente (animales, peatones, otro accidente, etc).
10. ``Police_Force``: Tipo de policía que acude al lugar del accidente.
11. ``Road_Surface_Conditions``: Condiciones de la carretera en el lugar del accidente.
12. ``Road_Type``: Tipo de carretera del lugar del accidente.
13. ``Time``: Hora del accidente.
14. ``Urban_or_Rural_Area``: Zona urbana o rural. 
15. ``Weather_Conditions``: Condiciones climáticas en el lugar del accidente (lluvia, viento, niebla, etc).
16. ``Vehicle_Type``: Tipo de vehículo que provoca el accidente.

<br>

Las variables de fecha y hora tienen que ser transformadas a una variable de tipo timestamp para manejarse mejor. Como la variable con la hora (``Time``) tiene datos faltantes, los rellenaremos por defecto con la hora '00:00', ya que no hay ninguna otra observación con esta hora y podrán seguir siendo identificables en caso de causar problemas.

La otra variable que también tiene NAs es ``Carriageway_Hazards``, para la cual el 98% de los datos son faltantes. Para no asumir ninguna cualidad sobre esta variable, los reemplazaremos por el valor por defecto 'unknown'.


#### Variables numéricas

1. ``Latitude``: Latitud de la ubicación geográfica del accidente.
2. ``Longitude``: Longitud de la ubicación geográfica del accidente.
3. ``Number_of_Casualties``: Número de víctimas involucradas en el accidente.
4. ``Number_of_Vehicles``: Número de vehículos involucrados en el accidente.
5. `Speed_limit`: Límite de velocidad en el lugar del accidente.

Ninguna de estas variables tiene datos faltantes.

# 2. Limpieza y preprocesamiento

En esta sección, ...

## 2.1. Eliminación de valores faltantes

In [91]:
# Eliminar NAs de variable de hora
accidents["Time"] = accidents["Time"].fillna("00:00")

# Crear variable de marca de tiempo
accidents["timestamp"] = pd.to_datetime(accidents["Accident Date"] + ' ' + accidents["Time"], format = "%d-%m-%Y %H:%M")

# Rango temporal de los accidentes del dataset
print(f"Rango temporal:{accidents['timestamp'].min(): %B %d, %Y %H:%M} <-->" + \
      f"{accidents['timestamp'].max(): %B %d, %Y %H:%M}")

# Eliminar NAs de variable Carriageway_Hazards
accidents["Carriageway_Hazards"] = accidents["Carriageway_Hazards"].fillna("unknown")

Rango temporal: January 01, 2021 00:01 <--> December 31, 2022 23:30


## 2.2. Corrección de errores

Cadenas de texto con errores

# 3. Análisis exploratorio

# 3.x. Atípicos

# 4. Estimación de factores de riesgo

De accidentes graves en comparación con accidentes leves


## 4.1. Metodología

Transformaciones, modelo, etc


## 4.2. Ajuste


## 4.3. Validación


## 4.4. Conclusiones

# 5. Comparativa de riesgo entre regiones



## 5.1. Metodología

## 5.2. Ajuste

## 5.3. Validación

## 5.4. Conclusiones

# 6. Conclusiones